## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tuatapere,NZ,-46.1333,167.6833,53.82,83,99,2.04,overcast clouds
1,1,Dikson,RU,73.5069,80.5464,-4.49,91,100,25.28,overcast clouds
2,2,Saint-Philippe,RE,-21.3585,55.7679,83.79,79,100,13.67,light rain
3,3,Severo-Kurilsk,RU,50.6789,156.1250,20.75,70,19,3.87,few clouds
4,4,Cape Town,ZA,-33.9258,18.4232,74.88,76,0,17.27,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Saint-Philippe,RE,-21.3585,55.7679,83.79,79,100,13.67,light rain
5,5,Hobart,AU,-42.8794,147.3294,78.22,49,75,16.11,broken clouds
8,8,Saint-Pierre,RE,-21.3393,55.4781,83.88,70,0,9.22,clear sky
11,11,Rikitea,PF,-23.1203,-134.9692,79.20,70,0,16.55,clear sky
14,14,Puro,PH,12.4841,123.3827,82.02,73,100,14.34,overcast clouds
21,21,Port Alfred,ZA,-33.5906,26.8910,75.51,85,100,19.10,overcast clouds
22,22,Vaini,TO,-21.2000,-175.2000,82.56,83,75,6.91,broken clouds
27,27,Binucayan,PH,8.2500,125.7167,85.03,65,69,2.37,broken clouds
31,31,Carnarvon,AU,-24.8667,113.6333,89.67,58,48,17.27,scattered clouds
32,32,Quatre Cocos,MU,-20.2078,57.7625,86.11,77,17,13.98,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                172
City                   172
Country                171
Lat                    172
Lng                    172
Max Temp               172
Humidity               172
Cloudiness             172
Wind Speed             172
Current Description    172
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows
preferred_cities_df.dropna(how = "any",subset = ["Country"], inplace = True)

preferred_cities_df.count()

C:\Users\opahu\anaconda3\envs\Pythondata\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


City_ID                171
City                   171
Country                171
Lat                    171
Lng                    171
Max Temp               171
Humidity               171
Cloudiness             171
Wind Speed             171
Current Description    171
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Saint-Philippe,RE,83.79,light rain,-21.3585,55.7679,
5,Hobart,AU,78.22,broken clouds,-42.8794,147.3294,
8,Saint-Pierre,RE,83.88,clear sky,-21.3393,55.4781,
11,Rikitea,PF,79.20,clear sky,-23.1203,-134.9692,
14,Puro,PH,82.02,overcast clouds,12.4841,123.3827,
21,Port Alfred,ZA,75.51,overcast clouds,-33.5906,26.8910,
22,Vaini,TO,82.56,broken clouds,-21.2000,-175.2000,
27,Binucayan,PH,85.03,broken clouds,8.2500,125.7167,
31,Carnarvon,AU,89.67,scattered clouds,-24.8667,113.6333,
32,Quatre Cocos,MU,86.11,few clouds,-20.2078,57.7625,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

hotel_df.head()

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Saint-Philippe,RE,83.79,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
5,Hobart,AU,78.22,broken clouds,-42.8794,147.3294,Mantra on Collins Hobart
8,Saint-Pierre,RE,83.88,clear sky,-21.3393,55.4781,Lindsey Hôtel
11,Rikitea,PF,79.20,clear sky,-23.1203,-134.9692,People ThankYou
14,Puro,PH,82.02,overcast clouds,12.4841,123.3827,


In [17]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Saint-Philippe,RE,83.79,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
5,Hobart,AU,78.22,broken clouds,-42.8794,147.3294,Mantra on Collins Hobart
8,Saint-Pierre,RE,83.88,clear sky,-21.3393,55.4781,Lindsey Hôtel
11,Rikitea,PF,79.20,clear sky,-23.1203,-134.9692,People ThankYou
21,Port Alfred,ZA,75.51,overcast clouds,-33.5906,26.8910,The Halyards Hotel
22,Vaini,TO,82.56,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
31,Carnarvon,AU,89.67,scattered clouds,-24.8667,113.6333,Hospitality Carnarvon
32,Quatre Cocos,MU,86.11,few clouds,-20.2078,57.7625,Sunrise Attitude Hotel
43,Cuamba,MZ,75.92,light rain,-14.8031,36.5372,Hotel Vision
44,Esperance,AU,76.84,clear sky,-33.8667,121.9000,Hospitality Esperance


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))